In [ ]:
# Importación de librerías necesarias para el proyecto

# Automatización de navegadores (Selenium)
# -----------------------------------------------------------------------
from selenium import webdriver                      # Control automático del navegador para realizar scraping web
from selenium.webdriver.chrome.service import Service  # Manejo del servicio de ChromeDriver
from selenium.webdriver.common.by import By         # Selección de elementos en el DOM
from selenium.webdriver.chrome.options import Options  # Configuración de opciones del navegador (headless mode, etc.)
import time                                         # Gestión de pausas y temporización
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd                                 # Manipulación y análisis de datos en estructuras DataFrame
import numpy as np                                  # Cálculos numéricos y manejo de arrays
import datetime                                     # Manipulación de fechas y tiempos

# Visualización de progreso
# -----------------------------------------------------------------------
from tqdm import tqdm                               # Creación de barras de progreso durante bucles

# Manejo de tiempo y solicitudes con espera
# -----------------------------------------------------------------------
from time import sleep                              # Pausas en la ejecución del código

import sys
sys.path.append("../")
from src.soporte_scraping import (
    df_productos
)

# Webs a Scrapear
- https://minimalismbrand.com/collections/sweater-men
- "https://minimalismbrand.com/collections/sudaderas"

In [ ]:
url_scrap = ["https://minimalismbrand.com/collections/sudaderas","https://minimalismbrand.com/collections/sweater-men"]
df = df_productos(url_scrap)

In [ ]:
df.head()

In [ ]:
df.to_csv("../datos/01_minimalism_productos_sin_info.csv")

### Unir lo de Nelson

In [ ]:
df = pd.read_csv("../datos/01_minimalism_productos_sin_info.csv",index_col=0)

In [ ]:
df_info = pd.read_csv("../datos/02_info_prendas.csv",index_col=0)
df_info.sample()

In [ ]:
df_info.rename(columns={'nombre': 'producto'}, inplace=True)
columns_to_add = ['precio', 'descripción', 'Fabricación (materia prima + confección + mano de obra)',
                  'Operaciones (logística + transporte)', 'Sueldos equipo + marketing + comunicación',
                  'Margen antes de impuestos']

df = pd.merge(df, df_info[['producto'] + columns_to_add], on='producto', how='left')


In [ ]:
df.sample()

### Limpiar tabla

In [ ]:
df = df.drop(columns="descripción")
df.sample()

Renombramos columnas

In [ ]:
df = df.rename(columns={"precio":"precio (€)",
                        "Fabricación (materia prima + confección + mano de obra)":"coste fabricacion (%)",
                        "Operaciones (logística + transporte)":"coste logistica y transporte (%)",
                        "Sueldos equipo + marketing + comunicación":"coste sueldos equipo + marketing + comunicacion (%)",
                        "Margen antes de impuestos":"margen antes de impuestos (%)"})

In [ ]:
df.sample()

Eliminamos € de precio y % de las columnas

In [ ]:
df["precio (€)"] = df["precio (€)"].str.strip("€")
df.sample()

In [ ]:
df["coste fabricacion (%)"] = df["coste fabricacion (%)"].str.strip("%")
df["coste logistica y transporte (%)"] = df["coste logistica y transporte (%)"].str.strip("%")
df["coste sueldos equipo + marketing + comunicacion (%)"] = df["coste sueldos equipo + marketing + comunicacion (%)"].str.strip("%")
df["margen antes de impuestos (%)"] = df["margen antes de impuestos (%)"].str.strip("%")
df.sample()


Ajustar formatos a float e int

In [ ]:
df.info()

In [ ]:
df["precio (€)"] = df["precio (€)"].astype(float)
df["coste fabricacion (%)"] = df["coste fabricacion (%)"].astype(float)
df["coste logistica y transporte (%)"] = df["coste logistica y transporte (%)"].astype(float)
df["coste sueldos equipo + marketing + comunicacion (%)"] = df["coste sueldos equipo + marketing + comunicacion (%)"].astype(float)
df["margen antes de impuestos (%)"] = df["margen antes de impuestos (%)"].astype(float)

In [ ]:
df.sample()

In [ ]:
df.info()

In [ ]:
df.to_csv("../datos/03_productos_minimalism.csv")

# Competencia
- https://thinkingmu.com/

In [123]:
driver = webdriver.Chrome()
driver.get("https://thinkingmu.com/")
driver.maximize_window()
sleep(1)
driver.find_element("xpath","/html/body/div[1]/div/div[1]/div/div[2]/button[3]").click()
sleep(1)
driver.execute_script("window.scrollBy(0, 1500);")
driver.find_element("xpath","/html/body/main/div[2]/div/div[2]/div[2]/a[2]").click()
sleep(3)
driver.find_element("xpath","/html/body/main/div/div/div[2]/div/ul/li[7]/label/span").click()
sleep(1)
productos = driver.find_elements("xpath","/html/body/main/div/div/div[3]/div/ul/li")
lis_producto = []
lis_precio = []
lis_material = []
lis_envio = []
for producto in productos:
    try:
        producto.click()
        sleep(1)
        try:
            producto = driver.find_element("xpath","/html/body/main/section[1]/section/div/div[2]/h1").text
            lis_producto.append(producto)
            sleep(0.5)
            precio = driver.find_element("xpath","/html/body/main/section[1]/section/div/div[2]/div[1]/div/div/div[1]/span").text
            lis_precio.append(precio)
            sleep(1)
            try:
                driver.find_element("css selector","#Product-meta-info-template--22527716622662__main > div.meta-toggles.meta-toggles-desktop > div:nth-child(1)")
            except:
                try:
                    driver.find_element("css selector","#Product-meta-info-template--22527716655430__main > div.meta-toggles.meta-toggles-desktop > div.meta-toggle.expanded > div.toggle.noselect").click()
                except:
                    driver.find_element("css selector","#Product-meta-info-template--22527716655430__main > div.meta-toggles.meta-toggles-desktop > div:nth-child(2)").click()
            
            try:
                element = driver.find_element("css selector","#Product-meta-info-template--22527716655430__f4048341-92c6-495b-833b-9f1f1b524e3f")
            except:
                element = driver.find_element("css selector","#Product-meta-info-template--22527716622662__f4048341-92c6-495b-833b-9f1f1b524e3f > div.blue-cardmodule > div > div")

            driver.execute_script("arguments[0].scrollIntoView();", element)
            sleep(1)
            driver.execute_script("window.scrollBy(0, -300);")
            sleep(1)
            try:
                driver.find_element("xpath","/html/body/main/section[1]/section/div/div[2]/div[8]/div[1]/div[3]/div[1]").click()
            except:
                driver.find_element("css selector","#Product-meta-info-template--22527716655430__main > div.meta-toggles.meta-toggles-desktop > div:nth-child(3) > div.toggle.noselect").click()
            try:
                envio = driver.find_element("xpath", "/html/body/main/section[1]/section/div/div[2]/div[8]/div[1]/div[3]/div[2]/p[2]").text
                envio = envio.split("\n")[1].split(": ")[1]
                lis_envio.append(envio)
            except:
                envio = driver.find_element("css selector", "#Product-meta-info-template--22527716655430__main > div.meta-toggles.meta-toggles-desktop > div.meta-toggle.expanded > div.content > p:nth-child(2)").text
                envio = envio.split("\n")[1].split(": ")[1]
                lis_envio.append(envio)
            driver.back()
        except:
            mice = driver.find_element("xpath","/html/body/div[15]/div/div[2]")
            mice.find_element("xpath","/html/body/div[15]/div/div[2]/div/div/div/div/div/button").click()
    except:
        mice = driver.find_element("xpath","/html/body/div[15]/div/div[2]")
        mice.find_element("xpath","/html/body/div[15]/div/div[2]/div/div/div/div/div/button").click()

lis_material.append("Exterior: 100% Poliéster Reciclado Forro: 100% Poliéster Reciclado")
lis_material.append("100% Algodón Orgánico")
lis_material.append("100% Algodón Orgánico")
lis_material.append("55% Cáñamo + 45% Algodón Orgánico")
lis_material.append("55% Cáñamo + 45% Algodón Orgánico")
lis_material.append(f"99% Algodón Orgánico + 1% Elastano")

In [124]:
lis_producto

['Chaqueta navy Jackson',
 'Sobrecamisa gris pana Theo',
 'Sobrecamisa beige pana Theo',
 'Chaqueta azul Carmy',
 'Chaqueta marrón Carmy',
 'Chaqueta dark clean denim Odonata']

In [127]:
lis_material

['Exterior: 100% Poliéster Reciclado Forro: 100% Poliéster Reciclado',
 '100% Algodón Orgánico',
 '100% Algodón Orgánico',
 '55% Cáñamo + 45% Algodón Orgánico',
 '55% Cáñamo + 45% Algodón Orgánico',
 '99% Algodón Orgánico + 1% Elastano']

In [126]:
lis_precio

['159,90 €', '119,90 €', '119,90 €', '149,90 €', '149,90 €', '129,90 €']

In [131]:
df = pd.DataFrame({
    "producto":lis_producto,
    "precio":lis_precio,
    "material":lis_material,
    "envio":lis_envio,
    "tallas": [["S","M","L","XL","XXL"]]*6
})

In [132]:
df

,producto,precio,material,envio,tallas
0,Chaqueta navy Jackson,"159,90 €",Exterior: 100% Poliéster Reciclado Forro: 100%...,Entrega en 1-2 días hábiles. Envío gratuito a ...,"[S, M, L, XL, XXL]"
1,Sobrecamisa gris pana Theo,"119,90 €",100% Algodón Orgánico,Entrega en 1-2 días hábiles. Envío gratuito a ...,"[S, M, L, XL, XXL]"
2,Sobrecamisa beige pana Theo,"119,90 €",100% Algodón Orgánico,Entrega en 1-2 días hábiles. Envío gratuito a ...,"[S, M, L, XL, XXL]"
3,Chaqueta azul Carmy,"149,90 €",55% Cáñamo + 45% Algodón Orgánico,Entrega en 1-2 días hábiles. Envío gratuito a ...,"[S, M, L, XL, XXL]"
4,Chaqueta marrón Carmy,"149,90 €",55% Cáñamo + 45% Algodón Orgánico,Entrega en 1-2 días hábiles. Envío gratuito a ...,"[S, M, L, XL, XXL]"
5,Chaqueta dark clean denim Odonata,"129,90 €",99% Algodón Orgánico + 1% Elastano,Entrega en 1-2 días hábiles. Envío gratuito a ...,"[S, M, L, XL, XXL]"


In [134]:
df["precio"] = df["precio"].str.strip(" €").str.replace(",",".").astype(float)

In [136]:
df = df.rename(columns={"precio":"precio (€)"})

In [ ]:
df.to_csv("../datos/04_productos_thinkingMu.csv")

# Sepiia

In [151]:
df = pd.read_csv("../datos/05_producto_sepia.csv",index_col=0)
df

,nombre,precio,descripción,entregas
0,AIR COAT AZUL MARINO,"299,00€",Fibras innovadoras de poliéster reciclado anti...,→ Entrega en 24-48 horas


In [152]:
df = df.rename(columns={"precio":"precio (€)"})
df["precio (€)"] = df["precio (€)"].str.strip("€").str.replace(",",".").astype(float)
df["entregas"] = "Entrega en 24-48 horas"
df

,nombre,precio (€),descripción,entregas
0,AIR COAT AZUL MARINO,299.0,Fibras innovadoras de poliéster reciclado anti...,Entrega en 24-48 horas


In [155]:
df["tallas"] = [["S","M","L","XL","XXL"]]

In [158]:
df.to_csv("../datos/07_datos_sepiia_limpios.csv")